# Введение в искусственные нейронные сети
# Урок 3. TensorFlow

### Нейронная сеть, которая может вычислять настроение авторов кинорецензий 

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers

Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Dropout

#### Загрузим датесет IMDb, который уже встроен в Keras. 

**Так как нам не подходят пропорции 50/50 данных обучения и тестирования, мы объединим их для 
последующего разделения в пропорции 80/20**

In [4]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

#### Изучение данных

In [5]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))

Categories: [0 1]
Number of unique words: 9998


In [6]:
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 234.75892
Standard Deviation: 173.0


**У нас имеются две категории: 0 или 1, что представляет собой настроение обзора. Весь датасет содержит 9998 уникальных слов, средний размер обзора составляет 234 слова со стандартным отклонением 173.**

In [7]:
print("Label:", targets[0])

Label: 1


In [8]:
print(data[2])

[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5974, 54, 61, 369, 13, 71, 149, 14, 22, 112, 4, 2401, 311, 12, 16, 3711, 33, 75, 43, 1829, 296, 4, 86, 320, 35, 534, 19, 263, 4821, 1301, 4, 1873, 33, 89, 78, 12, 66, 16, 4, 360, 7, 4, 58, 316, 334, 11, 4, 1716, 43, 645, 662, 8, 257, 85, 1200, 42, 1228, 2578, 83, 68, 3912, 15, 36, 165, 1539, 278, 36, 69, 2, 780, 8, 106, 14, 6905, 1338, 18, 6, 22, 12, 215, 28, 610, 40, 6, 87, 326, 23, 2300, 21, 23, 22, 12, 272, 40, 57, 31, 11, 4, 22, 47, 6, 2307, 51, 9, 170, 23, 595, 116, 595, 1352, 13, 191, 79, 638, 89, 2, 14, 9, 8, 106, 607, 624, 35, 534, 6, 227, 7, 129, 113]


In [9]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[2]] )# обратное преобразование индексов в слова
print(decoded)

# this has to be one of the worst films of the 1990s when my friends i were watching this film being the target audience it was aimed at we just sat watched the first half an hour with our jaws touching the floor at how bad it really was the rest of the time everyone else in the theatre just started talking to each other leaving or generally crying into their popcorn that they actually paid money they had # working to watch this feeble excuse for a film it must have looked like a great idea on paper but on film it looks like no one in the film has a clue what is going on crap acting crap costumes i can't get across how # this is to watch save yourself an hour a bit of your life


**Подготовка данных.**

**Самый большой обзор имеет около 10 000 слов, поэтому все обзоры приведем к такому же количеству слов, 
заполнив недостающие символы нулями**

In [10]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results
 
data = vectorize(data)
targets = np.array(targets).astype("float32")

**Разделим датасет на обучающий и тестировочный наборы. Обучающий набор будет состоять из 40 000 обзоров, тестировочный — из 10 000.**

In [11]:
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

In [12]:
train_y.shape


(40000,)

In [13]:
data[:10000].shape

(10000, 10000)

In [14]:
data[10000:].shape

(40000, 10000)

In [15]:
model = Sequential([
  # Input Layer  
  Dense(70, activation='elu', input_shape=(10000, )),
  # Hidden Layers  
  Dropout(0.7, noise_shape=None, seed=None),
  Dense(50, activation='elu'),
#   Dropout(0.5, noise_shape=None, seed=None),
#   Dense(170, activation='elu'),  
#   Dropout(0.5, noise_shape=None, seed=None),
#   Dense(120, activation='elu'),     
  # Output Layer, softmax тут не работает, использовал sigmoid
  # Dense(1, activation='softmax'), 
  Dense(1, activation='sigmoid'), 
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 70)                700070    
_________________________________________________________________
dropout_1 (Dropout)          (None, 70)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3550      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 703,671
Trainable params: 703,671
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compiling the model
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)
# Fit the model
results = model.fit(
 train_x, train_y,
 epochs= 2,
 batch_size = 500,
 validation_data = (test_x, test_y)
)
print("Test-Accuracy:", np.mean(results.history["val_accuracy"]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 4s 105us/step - loss: 0.3654 - accuracy: 0.8410 - val_loss: 0.2604 - val_accuracy: 0.8964
Epoch 2/2
40000/40000 [==============================] - 4s 97us/step - loss: 0.2240 - accuracy: 0.9130 - val_loss: 0.2623 - val_accuracy: 0.8974
Test-Accuracy: 0.8968999981880188


**Нейронная сеть получилась с четырьмя слоями. В скрытых слоях использована функция активации «elu». На каждом слое используется функция «dense» для полносвязного соединения слоев друг с другом. Использован оптимизатор «adam». Оптимизатор — это алгоритм, который изменяет веса и смещения во время обучения. В качестве функции потерь используем бинарную кросс-энтропию (так как мы работаем с бинарной классификацией), в качестве метрики оценки — точность.**

Модель можно использовать для анализа бинарных настроений в других источниках, но для этого придется сделать их размер равным 10 000 или изменить параметры входного слоя.